# Lab | Langchain Evaluation

## Intro

Pick different sets of data and re-run this notebook. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications.

What did you learn? - Let's discuss that in class

## LangChain: Evaluation

### Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [101]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY') 

In [102]:
import os
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

# Load path from the environment variable
env_ih1 = os.getenv("ENV_IH1")

dotenv_path = Path(env_ih1)
load_dotenv(dotenv_path=dotenv_path)

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY= os.getenv('PINECONE_KEY')

### Example 1

#### Create our QandA application

In [103]:
!pip install langchain_huggingface

Defaulting to user installation because normal site-packages is not writeable


In [104]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import LLMChain


In [105]:
file = './data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [106]:
# !pip install --upgrade --force-reinstall sentence-transformers

In [107]:
!pip install docarray

Defaulting to user installation because normal site-packages is not writeable


In [108]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cuda'})
).from_loaders([loader])

In [109]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

#### Coming up with test datapoints

In [110]:
data[10]

Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\r\n\r\nSize & Fit\r\n- Pants are Favorite Fit: Sits lower on the waist.\r\n- Relaxed Fit: Our most generous fit sits farthest from the body.\r\n\r\nFabric & Care\r\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\r\n\r\nAdditional Features\r\n- Relaxed fit top with raglan sleeves and rounded hem.\r\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\r\n\r\nImported.")

In [111]:
data[11]

Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

#### Hard-coded examples

In [112]:
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field

examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="Examples:\n"
             "1. Query: Do the Cozy Comfort Pullover Set have side pockets?\n"
             "   Answer: Yes\n"
             "2. Query: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?\n"
             "   Answer: The DownTek collection\n"
             "Query: {query}\n"
             "Answer:"
)

# Define the output model
class Answer(BaseModel):
    answer: str = Field(description="The answer to the query")

# Create the output parser
class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Answer:
        # Split the response to get the answer
        answer = text.strip().split("Answer:")[-1].strip()
        return Answer(answer=answer)

# Initialize the LLM
# llm = OpenAI()
llm = ChatOpenAI()

# Create the LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_parser=AnswerOutputParser()
)

# Example query
query = "What could I wear to look?"

# Run the chain
result = llm_chain.run({"query": query})

# Print the result
print(result)


answer='It depends on the occasion and your personal style, but you could consider wearing a sleek blazer with dress pants for a formal look, a stylish dress or blouse with jeans for a casual look, or a statement outfit that reflects your unique personality.'


In [113]:
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field

# Define examples
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

context = """you are a sassy Italian fashion personal shopper who speaks with an Italian accent. All your answer should relate to the databse. Else answer 'I don't think we can darling" \n"""

# Format the examples into a string
formatted_examples = "\n".join(
    [f"Context: {context}\nQuery: {example['query']}\nAnswer: {example['answer']}" for example in examples]
)

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template=f"Examples:\n{formatted_examples}\nQuery: {{query}}\nAnswer:"
)

# Define the output model
class Answer(BaseModel):
    answer: str = Field(description="The answer to the query")

# Create the output parser
class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Answer:
        # Split the response to get the answer
        answer = text.strip().split("Answer:")[-1].strip()
        return Answer(answer=answer)

# Initialize the LLM
# llm = OpenAI()
llm = ChatOpenAI()

# Create the LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_parser=AnswerOutputParser()
)

# Example query
query = "I need somehting cozy and not too sexy. What can you advice?"

# Run the chain
result = llm_chain.run({"query": query})

# Print the result
print(result)


answer="How about the Cozy Comfort Pullover Set? It's both cozy and stylish."


#### LLM-Generated examples

In [114]:
from langchain.evaluation.qa import QAGenerateChain

In [115]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [116]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

In [117]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

C:\Users\aurel\AppData\Roaming\Python\Python312\site-packages\langchain\chains\llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [118]:
new_examples[0]

{'qa_pairs': {'query': "What is the weight of a pair of Women's Campside Oxfords?",
  'answer': "The approximate weight of a pair of Women's Campside Oxfords is 1 lb. 1 oz."}}

In [119]:
data[0]

Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.")

In [120]:
d_flattened = [data['qa_pairs'] for data in new_examples]
d_flattened

[{'query': "What is the weight of a pair of Women's Campside Oxfords?",
  'answer': "The approximate weight of a pair of Women's Campside Oxfords is 1 lb. 1 oz."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What are some key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece as described in the document?",
  'answer': "The key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece include bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bottom for secure fit and maximum coverage, and is recommended for machine wash and line dry."},
 {'query': 'What is the fabric composition of the Refresh Swimwear V-Neck Tankini Contra

#### Combine examples

In [121]:
# examples += new_example
examples += d_flattened

In [122]:
examples[0]

{'query': 'Do the Cozy Comfort Pullover Set have side pockets?',
 'answer': 'Yes'}

In [123]:
qa.invoke(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set has side seam pockets and a back zip pocket, as well as two elastic mesh water bottle pockets.'}

### Manual Evaluation - Fun part

In [124]:
import langchain
langchain.debug = True

In [125]:
qa.invoke(examples[1]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?",
  "context": ": 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, loft

{'query': 'What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?',
 'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'}

In [126]:
# Turn off the debug mode
langchain.debug = False

### LLM assisted evaluation

In [127]:
examples += d_flattened

In [128]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What is the weight of a pair of Women's Campside Oxfords?",
  'answer': "The approximate weight of a pair of Women's Campside Oxfords is 1 lb. 1 oz."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What are some key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece as described in the document?",
  'answer': "The key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece include bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no

In [129]:
predictions = qa.batch(examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [130]:
predictions

[{'query': 'Do the Cozy Comfort Pullover Set have side pockets?',
  'answer': 'Yes',
  'result': 'Yes, the Cozy Comfort Pullover Set has side seam pockets and a back zip pocket, as well as two elastic mesh water bottle pockets.'},
 {'query': 'What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection',
  'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'},
 {'query': "What is the weight of a pair of Women's Campside Oxfords?",
  'answer': "The approximate weight of a pair of Women's Campside Oxfords is 1 lb. 1 oz.",
  'result': "The weight of a pair of Women's Campside Oxfords is approximately 1 lb. 1 oz."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".',
  'result': 'The dimensions of the small size of the Recycled Wat

In [131]:
from langchain.evaluation.qa import QAEvalChain

In [132]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [133]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [134]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [135]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    # print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set has side seam pockets and a back zip pocket, as well as two elastic mesh water bottle pockets.

Example 1:
Question: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.

Example 2:
Question: What is the weight of a pair of Women's Campside Oxfords?
Real Answer: The approximate weight of a pair of Women's Campside Oxfords is 1 lb. 1 oz.
Predicted Answer: The weight of a pair of Women's Campside Oxfords is approximately 1 lb. 1 oz.

Example 3:
Question: What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?
Real Answer: The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".
Predicted Answer: The dimensions of 

### Example 2
One can also easily evaluate your QA chains with the metrics offered in ragas

In [136]:
from langchain_huggingface import HuggingFaceEmbeddings
loader = TextLoader("./data/nyc_text.txt")
index = VectorstoreIndexCreator(embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cuda'})).from_loaders([loader])


llm = ChatOpenAI(temperature= 0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=index.vectorstore.as_retriever(),
    return_source_documents=True,
)

C:\Users\aurel\AppData\Roaming\Python\Python312\site-packages\langchain\indexes\vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [155]:
# testing it out

question = "How did New York City get its name?"
result = qa_chain.invoke({"query": question})
result["result"]

'New York City was originally founded as a Dutch trading post called New Amsterdam in 1626. In 1664, the city came under British control and was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city has been continuously named New York since November 1674.'

In [138]:
result

{'query': 'How did New York City get its name?',
 'result': 'New York City was originally named New Amsterdam by Dutch colonists in 1626. When the city came under British control in 1664, it was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city has been continuously named New York since November 1674.',
 'source_documents': [Document(id='61f99851-2825-4b6c-9db0-86fff6bd32da', metadata={'source': './data/nyc_text.txt'}, page_content='The city and its metropolitan area constitute the premier gateway for legal immigration to the United States. As many as 800 languages are spoken in New York, making it the most linguistically diverse city in the world. New York City is home to more than 3.2 million residents born outside the U.S., the largest foreign-born population of any city in the world as of 2016.New York City traces its origins to a trading post founded on the southern tip of Manhattan Island by Dutch colonists in approxima

Now in order to evaluate the qa system we generated a few relevant questions. We've generated a few question for you but feel free to add any you want.

In [139]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
    "What's the relationship between New York City and the French city of Angouleme?"
]

eval_answers = [
    "8,804,190",
    "Brooklyn",
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
    "New York City and the French city of Angoulême share a historical connection dating back to the early 16th century. In 1524, the Italian explorer Giovanni da Verrazzano, commissioned by King Francis I of France, became the first European to enter what is now New York Harbor. In honor of King Francis I, who held the title of Count of Angoulême before ascending to the throne, Verrazzano named the newly discovered land 'Nouvelle-Angoulême' (New Angoulême).",
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [140]:
examples

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'ground_truths': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'ground_truths': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City

#### Introducing RagasEvaluatorChain

`RagasEvaluatorChain` creates a wrapper around the metrics ragas provides (documented [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)), making it easier to run these evaluation with langchain and langsmith.

The evaluator chain has the following APIs

- `__call__()`: call the `RagasEvaluatorChain` directly on the result of a QA chain.
- `evaluate()`: evaluate on a list of examples (with the input queries) and predictions (outputs from the QA chain). 
- `evaluate_run()`: method implemented that is called by langsmith evaluators to evaluate langsmith datasets.

lets see each of them in action to learn more.

In [141]:
result = qa_chain.invoke({"query": eval_questions[5]})
result["result"]

'New York City was originally claimed by the French explorer Giovanni da Verrazzano in 1524 and named Nouvelle Angoulême (New Angoulême) in honor of the French region of Angoulême. This historical connection is reflected in the naming of New York City.'

In [144]:
key_mapping = {
    "query": "question",
    "result": "answer",
    "source_documents": "contexts",
    "ground_truth": "contexts"
}

result_updated = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        result_updated[new_key] = result[old_key]


In [147]:
result_updated

{'question': "What's the relationship between New York City and the French city of Angouleme?",
 'answer': 'New York City was originally claimed by the French explorer Giovanni da Verrazzano in 1524 and named Nouvelle Angoulême (New Angoulême) in honor of the French region of Angoulême. This historical connection is reflected in the naming of New York City.',
 'contexts': [Document(id='155d0155-d228-40bb-83f9-eb6b834c1e4c', metadata={'source': './data/nyc_text.txt'}, page_content="In the pre-Columbian era, the area of present-day New York City was inhabited by Algonquian Native Americans, including the Lenape. Their homeland, known as Lenapehoking, included the present-day areas of Staten Island, Manhattan, the Bronx, the western portion of Long Island (including the areas that would later become the boroughs of Brooklyn and Queens), and the Lower Hudson Valley.The first documented visit into New York Harbor by a European was in 1524 by Italian Giovanni da Verrazzano, an explorer from 

In [44]:
# !pip install --no-cache-dir recordclass

In [45]:
# !pip install ragas==0.1.9

In [148]:
from ragas.integrations.langchain import EvaluatorChain 
# from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)

# create evaluation chains
faithfulness_chain   = EvaluatorChain(metric=faithfulness)
answer_rel_chain     = EvaluatorChain(metric=answer_relevancy)
context_rel_chain    = EvaluatorChain(metric=context_relevancy)
context_recall_chain = EvaluatorChain(metric=context_recall)

1. `__call__()`

Directly run the evaluation chain with the results from the QA chain. Do note that metrics like context_relevancy and faithfulness require the `source_documents` to be present.

In [149]:
# Recheck the result that we are going to validate.
result

{'query': "What's the relationship between New York City and the French city of Angouleme?",
 'result': 'New York City was originally claimed by the French explorer Giovanni da Verrazzano in 1524 and named Nouvelle Angoulême (New Angoulême) in honor of the French region of Angoulême. This historical connection is reflected in the naming of New York City.',
 'source_documents': [Document(id='155d0155-d228-40bb-83f9-eb6b834c1e4c', metadata={'source': './data/nyc_text.txt'}, page_content="In the pre-Columbian era, the area of present-day New York City was inhabited by Algonquian Native Americans, including the Lenape. Their homeland, known as Lenapehoking, included the present-day areas of Staten Island, Manhattan, the Bronx, the western portion of Long Island (including the areas that would later become the boroughs of Brooklyn and Queens), and the Lower Hudson Valley.The first documented visit into New York Harbor by a European was in 1524 by Italian Giovanni da Verrazzano, an explorer 

In [150]:
print(type(result_updated))
print(result_updated)


<class 'dict'>
{'question': "What's the relationship between New York City and the French city of Angouleme?", 'answer': 'New York City was originally claimed by the French explorer Giovanni da Verrazzano in 1524 and named Nouvelle Angoulême (New Angoulême) in honor of the French region of Angoulême. This historical connection is reflected in the naming of New York City.', 'contexts': [Document(id='155d0155-d228-40bb-83f9-eb6b834c1e4c', metadata={'source': './data/nyc_text.txt'}, page_content="In the pre-Columbian era, the area of present-day New York City was inhabited by Algonquian Native Americans, including the Lenape. Their homeland, known as Lenapehoking, included the present-day areas of Staten Island, Manhattan, the Bronx, the western portion of Long Island (including the areas that would later become the boroughs of Brooklyn and Queens), and the Lower Hudson Valley.The first documented visit into New York Harbor by a European was in 1524 by Italian Giovanni da Verrazzano, an e

**Faithfulness**

In [ ]:
# source_texts = [doc.page_content for doc in result['source_documents']]
# eval_input = {
# 	"question": result["query"],
# 	"answer": result["result"],
# 	"contexts": source_texts
# }
# eval_result = faithfulness_chain(eval_input)
# eval_result["faithfulness_score"]


In [ ]:
# print(type(eval_input))
# print(eval_input)


NameError: name 'eval_input' is not defined

In [ ]:
### ORIGINAL BROKEN CODE
# eval_result = faithfulness_chain(result_updated)
# eval_result["faithfulness_score"]

1.0


In [151]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

# Extract text content from Document objects
result_updated["contexts"] = [doc.page_content for doc in result["source_documents"]]

eval_result = await faithfulness_chain.arun(result_updated)
print(result_updated)
print(eval_result)

{'question': "What's the relationship between New York City and the French city of Angouleme?", 'answer': 'New York City was originally claimed by the French explorer Giovanni da Verrazzano in 1524 and named Nouvelle Angoulême (New Angoulême) in honor of the French region of Angoulême. This historical connection is reflected in the naming of New York City.', 'contexts': ["In the pre-Columbian era, the area of present-day New York City was inhabited by Algonquian Native Americans, including the Lenape. Their homeland, known as Lenapehoking, included the present-day areas of Staten Island, Manhattan, the Bronx, the western portion of Long Island (including the areas that would later become the boroughs of Brooklyn and Queens), and the Lower Hudson Valley.The first documented visit into New York Harbor by a European was in 1524 by Italian Giovanni da Verrazzano, an explorer from Florence in the service of the French crown. He claimed the area for France and named it Nouvelle Angoulême (Ne

High faithfulness_score means that there are exact consistency between the source documents and the answer.

You can check lower faithfulness scores by changing the result (answer from LLM) or source_documents to something else.

In [152]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

# # Extract text content from Document objects
# result_updated["contexts"] = [doc.page_content for doc in result["source_documents"]]

fake_result = {
    "question": "How did New York City get its name?",
    "answer": "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "contexts": [
        "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor."
    ]
}

fake_eval_result = await faithfulness_chain.arun(fake_result)
print(eval_result)

0.0


**Context Relevancy**

In [154]:
eval_result = context_recall_chain(result_updated)
eval_result["context_recall_score"]

ValueError: Missing some input keys: {'ground_truth'}

High context_recall_score means that the ground truth is present in the source documents.

You can check lower context recall scores by changing the source_documents to something else.

In [156]:
from langchain.schema import Document
fake_result = result.copy()
fake_result["source_documents"] = [Document(page_content="I love christmas")]
eval_result = context_recall_chain(fake_result)
eval_result["context_recall_score"]

ValueError: Missing some input keys: {'contexts', 'ground_truth', 'question'}

2. `evaluate()`

Evaluate a list of inputs/queries and the outputs/predictions from the QA chain.

In [157]:
# run the queries as a batch for efficiency
predictions = qa_chain.batch(examples)

# evaluate
print("evaluating...")
r = faithfulness_chain.evaluate(examples, predictions)
r

evaluating...


AttributeError: 'EvaluatorChain' object has no attribute 'evaluate'

In [ ]:
# evaluate context recall
print("evaluating...")
r = context_recall_chain.evaluate(examples, predictions)
r